# Restaurant Finder - Interactive Chat (Free Tier APIs)

This notebook demonstrates the restaurant recommendation backend with a simple chat interface using **FREE TIER APIs only**.

Features:
- Search restaurants using Tavily (free tier) and Google Gemini (free tier) APIs
- Natural language queries (e.g., 'italian veg food in bangalore')
- Interactive chat loop
- Merged results from multiple sources

**Cost**: $0 - All APIs have generous free tiers perfect for students!

## Setup: Install Dependencies

In [ ]:
!uv pip install tavily-python google-generativeai python-dotenv -q

## Import Required Libraries

In [ ]:
import os
import json
import asyncio
from typing import Optional, List
from dotenv import load_dotenv
from tavily import AsyncTavilyClient
import google.generativeai as genai

# Load environment variables from root .env
load_dotenv('../../.env')

print("✅ Libraries imported successfully")

## Configure API Clients

In [ ]:
# Get API keys - all free tier!
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not TAVILY_API_KEY:
    raise ValueError("❌ TAVILY_API_KEY not found in environment")
if not GOOGLE_API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY not found in environment")

# Initialize API clients
tavily_client = AsyncTavilyClient(api_key=TAVILY_API_KEY)

# Initialize Gemini (free tier: 15 RPM, 1M TPM, 1500 RPD)
genai.configure(api_key=GOOGLE_API_KEY)
gemini_model = genai.GenerativeModel('gemini-2.5-flash-exp')

print("✅ API clients configured successfully")
print("📊 Free tier limits:")
print("   - Tavily: 1000 searches/month")
print("   - Gemini Flash: 15 RPM, 1M TPM, 1500 RPD")

## Define Search Functions

In [ ]:
async def search_tavily(query: str, location: Optional[str] = None) -> List[dict]:
    """Search restaurants using Tavily API"""
    try:
        search_query = f"{query} restaurants in {location}" if location else f"{query} restaurants"
        response = await tavily_client.search(
            query=search_query,
            search_depth="advanced",
            max_results=10
        )

        restaurants = []
        for result in response.get("results", []):
            restaurant = {
                "name": result.get("title", "Unknown Restaurant"),
                "address": result.get("url", "Address not available"),
                "cuisine": query,
                "rating": None,
                "description": result.get("content", "No description available")[:200],
                "source": "tavily"
            }
            restaurants.append(restaurant)

        return restaurants
    except Exception as e:
        print(f"❌ Tavily API Error: {str(e)}")
        return []

print("✅ Tavily search function defined")

In [ ]:
async def search_gemini(query: str, location: Optional[str] = None) -> List[dict]:
    """Search restaurants using Google Gemini API (free tier)"""
    try:
        search_query = f"{query} restaurants in {location}" if location else f"{query} restaurants"

        prompt = f"""You are a restaurant recommendation expert. Find restaurants for: {search_query}

Return restaurant data in JSON format with an array of restaurants, each having these fields:
- name (string)
- address (string)
- cuisine (string)
- rating (string or null)
- description (string)
- hours (string or null)
- price (string or null)
- phone (string or null)
- website (string or null)

Return at least 3-5 restaurants if available. Return ONLY the JSON array, no other text."""

        # Call Gemini API (synchronous, wrap in async)
        response = await asyncio.to_thread(
            gemini_model.generate_content,
            prompt
        )

        content = response.text

        # Try to extract JSON from response
        try:
            start = content.find('[')
            end = content.rfind(']') + 1
            if start != -1 and end > start:
                json_str = content[start:end]
                data = json.loads(json_str)
                
                restaurants = []
                for item in data:
                    restaurant = {
                        "name": item.get("name", "Unknown Restaurant"),
                        "address": item.get("address", "Address not available"),
                        "cuisine": item.get("cuisine", query),
                        "rating": item.get("rating"),
                        "description": item.get("description", "No description available"),
                        "hours": item.get("hours"),
                        "price": item.get("price"),
                        "phone": item.get("phone"),
                        "website": item.get("website"),
                        "source": "gemini"
                    }
                    restaurants.append(restaurant)
                
                return restaurants
        except json.JSONDecodeError:
            pass

        # Fallback
        return [{
            "name": "Restaurant suggestions",
            "address": "See description",
            "cuisine": query,
            "description": content[:500],
            "source": "gemini"
        }]

    except Exception as e:
        print(f"❌ Gemini API Error: {str(e)}")
        return []

print("✅ Gemini search function defined")

## Define Result Merging Function

In [ ]:
def merge_results(tavily_results: List[dict], gemini_results: List[dict]) -> List[dict]:
    """Merge and deduplicate results from both sources"""
    merged = []
    seen_names = set()

    # Prefer Gemini results first (more structured)
    for restaurant in gemini_results:
        name_lower = restaurant["name"].lower()
        if name_lower not in seen_names:
            seen_names.add(name_lower)
            merged.append(restaurant)

    # Add Tavily results if not duplicates
    for restaurant in tavily_results:
        name_lower = restaurant["name"].lower()
        if name_lower not in seen_names:
            seen_names.add(name_lower)
            merged.append(restaurant)

    return merged

print("✅ Merge function defined")

## Main Search Function

In [ ]:
async def search_restaurants(query: str, location: Optional[str] = None) -> List[dict]:
    """Search restaurants using both APIs and merge results"""
    print(f"\n🔍 Searching for: {query}" + (f" in {location}" if location else ""))
    
    # Search both APIs in parallel
    tavily_task = search_tavily(query, location)
    gemini_task = search_gemini(query, location)
    
    tavily_results, gemini_results = await asyncio.gather(
        tavily_task, gemini_task,
        return_exceptions=True
    )
    
    # Handle errors
    if isinstance(tavily_results, Exception):
        tavily_results = []
    if isinstance(gemini_results, Exception):
        gemini_results = []
    
    # Merge results
    merged = merge_results(
        tavily_results if isinstance(tavily_results, list) else [],
        gemini_results if isinstance(gemini_results, list) else []
    )
    
    return merged

print("✅ Main search function defined")

## Display Function

In [ ]:
def display_restaurant(restaurant: dict, index: int):
    """Display a single restaurant in a formatted way"""
    print(f"\n{'='*80}")
    print(f"🍽️  Restaurant #{index + 1}: {restaurant['name']}")
    print(f"{'='*80}")
    print(f"📍 Address: {restaurant['address']}")
    print(f"🍴 Cuisine: {restaurant['cuisine']}")
    
    if restaurant.get('rating'):
        print(f"⭐ Rating: {restaurant['rating']}")
    if restaurant.get('price'):
        print(f"💰 Price: {restaurant['price']}")
    if restaurant.get('hours'):
        print(f"🕒 Hours: {restaurant['hours']}")
    if restaurant.get('phone'):
        print(f"📞 Phone: {restaurant['phone']}")
    if restaurant.get('website'):
        print(f"🌐 Website: {restaurant['website']}")
    
    print(f"\n📝 Description: {restaurant['description']}")
    print(f"\n🔖 Source: {restaurant.get('source', 'unknown')}")

print("✅ Display function defined")

## Test Search (Single Query)

In [ ]:
# Test with a sample query
test_query = "italian veg food"
test_location = "bangalore"

results = await search_restaurants(test_query, test_location)

print(f"\n✅ Found {len(results)} restaurants!\n")

# Display first 3 results
for i, restaurant in enumerate(results[:3]):
    display_restaurant(restaurant, i)

## Interactive Chat Loop

Run this cell to start an interactive chat session. Type your restaurant queries and get recommendations!

**Example queries:**
- "chinese food in san francisco"
- "veg italian restaurants in mumbai"
- "best sushi places in tokyo"

Type `exit` or `quit` to stop.

In [ ]:
print("\n" + "="*80)
print("🍽️  RESTAURANT FINDER - Interactive Chat")
print("="*80)
print("\nAsk me to find restaurants! Examples:")
print("  - 'italian veg food in bangalore'")
print("  - 'chinese restaurants in san francisco'")
print("  - 'best sushi in tokyo'")
print("\nType 'exit' or 'quit' to stop.\n")

while True:
    query = input("\n🔍 Your query: ").strip()
    
    if query.lower() in ['exit', 'quit', 'q']:
        print("\n👋 Thanks for using Restaurant Finder! Goodbye!")
        break
    
    if not query:
        print("⚠️  Please enter a valid query.")
        continue
    
    # Parse query (simple approach - assume last word might be location)
    words = query.split()
    if len(words) >= 3 and words[-2].lower() == 'in':
        location = words[-1]
        cuisine = ' '.join(words[:-2])
    else:
        location = None
        cuisine = query
    
    # Search
    results = await search_restaurants(cuisine, location)
    
    if not results:
        print("\n❌ No restaurants found. Try a different query!")
        continue
    
    print(f"\n✨ Found {len(results)} restaurants!")
    
    # Ask how many to display
    try:
        num_display = int(input(f"\nHow many results to display? (1-{len(results)}, press Enter for 5): ") or "5")
        num_display = min(max(1, num_display), len(results))
    except ValueError:
        num_display = min(5, len(results))
    
    # Display results
    for i, restaurant in enumerate(results[:num_display]):
        display_restaurant(restaurant, i)

## Summary

This notebook demonstrates:
1. ✅ Integration with **Tavily (free tier)** and **Google Gemini (free tier)** APIs
2. ✅ Async parallel search for better performance
3. ✅ Result merging and deduplication
4. ✅ Interactive chat loop for continuous queries
5. ✅ Formatted restaurant display with all available information

**💰 Cost**: $0 - Perfect for students!
- Tavily: 1000 searches/month free
- Google Gemini Flash: 15 RPM, 1M TPM, 1500 RPD free

The backend logic matches the FastAPI implementation in `backend/app.py`.